# Import des modules

In [7]:
import numpy as np
import pandas as pd
# from utils.TSFEL_extraction import compute_global_tsfel
# from utils.TSFRESH_extraction import compute_global_tsfresh
from utils.load_data import transform,l_raw

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import cohen_kappa_score as CKS

from lightgbm import LGBMClassifier
from sklearn.metrics import make_scorer

# Extraction des features
### Avec TSFEL et TSFRESH

### Récupération des données et transformation

In [35]:
X_train, y_train, X_test, y_test = l_raw()

X_train=transform(X_train)
X_test=transform(X_test)

### Ajout des features

In [37]:
df_tsfel =pd.read_csv('DATA/TSFEL/tsfel_aug_diff.csv',index_col=0)
df_tsfresh = pd.read_csv('DATA/TSFRESH/tsfresh_aug.csv',index_col=0)

features_calculated = df_tsfel.join(df_tsfresh)
del df_tsfel
del df_tsfresh

In [31]:
###
# Baseline

clf = LGBMClassifier(class_weight='balanced', random_state=42,max_depth=8)
clf.fit(X_train,y_train)
sc_max = 0.4652731147636704

In [38]:
to_add=[]
for col in list(set(features_calculated.columns)-set(X_train.columns)):
    clf = LGBMClassifier(class_weight='balanced', random_state=42,max_depth=8)
    clf.fit(X_train.join(features_calculated[to_add+[col]]),y_train)
    sc = CKS(clf.predict(X_test.join(features_calculated[to_add+[col]])), y_test)
    if sc>sc_max:
        sc_max=sc
        to_add.append(col)
        print(sc,col,sep='\n')
        
X_train = X_train.join(features_calculated[to_add])
X_test = X_test.join(features_calculated[to_add])

KeyboardInterrupt: 

In [7]:
to_del=[]
for col in X_train:
    clf = LGBMClassifier(class_weight='balanced', random_state=42,max_depth=8)
    clf.fit(X_train.drop(to_del+[col],axis=1),y_train)
    sc = CKS(clf.predict(X_test.drop(to_del+[col],axis=1)), y_test)
    if sc>sc_max:
        sc_max=sc
        to_del.append(col)
        print(sc,col,sep='\n')
X_train=X_train.drop(to_del,axis=1)
X_test=X_test.drop(to_del,axis=1)

In [ ]:
clf = LGBMClassifier(class_weight='balanced',random_state=0,max_depth=8)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print("Cohen-Kappa",CKS(y_test, y_pred))

In [12]:
from core import Feature_generator
from sklearn.linear_model import RidgeClassifier

In [33]:
cll = make_pipeline(Scaler(),Feature_generator(RidgeClassifier(random_state=42),max_iter=50,scoring=make_scorer(CKS)),RidgeClassifier(random_state=42))
cll.fit(X_train,y_train)
y_pred=cll.predict(X_test)
print("Cohen-Kappa",CKS(y_test, y_pred))

  0%|                                                                                            | 0/50 [00:00<?, ?it/s][I 2022-11-11 14:22:08,573] A new study created in memory with name: no-name-38f9b792-c14e-4657-94d8-a9826b50f1f2


Q7batch nsumbatch


[I 2022-11-11 14:22:09,142] Trial 0 finished with value: -0.2190024626285704 and parameters: {'a': -1.364569996652354, 'b': 0.6601988074849854, 'a2': 0.5284318190601089, 'b2': -1.5909452098112262, 'a3': -1.0843540581716549, 'b3': 0.32814746005416184}. Best is trial 0 with value: -0.2190024626285704.
[I 2022-11-11 14:22:09,629] Trial 1 finished with value: -0.2427617076020943 and parameters: {'a': -1.0876073682041074, 'b': -0.35994527505035645, 'a2': 1.1136106573863178, 'b2': 1.7291701247367146, 'a3': 0.7040420606178186, 'b3': 0.7014520131173052}. Best is trial 1 with value: -0.2427617076020943.
[I 2022-11-11 14:22:10,307] Trial 2 finished with value: -0.22470627949283178 and parameters: {'a': 1.1641240380565838, 'b': 0.8932823558502916, 'a2': -0.20303605377758327, 'b2': 0.5816410660913061, 'a3': 0.3276320676959261, 'b3': 1.1821001501423472}. Best is trial 1 with value: -0.2427617076020943.
[W 2022-11-11 14:22:10,779] Trial 3 failed because of the following error: KeyboardInterrupt()
Tr

KeyboardInterrupt: 

In [11]:
cl2 = RidgeClassifier(random_state=42)
cl2.fit(X_train,y_train)
y_pred=cl2.predict(X_test)
print("Cohen-Kappa",CKS(y_test, y_pred))

Cohen-Kappa 0.15557833370703744


In [147]:
clf.fit(pd.concat([X_train,X_test]),pd.concat([y_train,y_test]))

LGBMClassifier(class_weight='balanced', max_depth=8, random_state=0)

In [156]:
test = transform(pd.read_csv('DATA/RAW/test.csv',index_col=0))
test = test.join(features_calculated[list(set(X_train)&set(features_calculated))])
test['TARGET'] = clf.predict(test)

In [158]:
test['TARGET'].to_csv('res.csv')